In [420]:


class PolynomialTerm:
    def __hash__(self):
        return hash(str(self))
    
    def __str__(self):
        return str(self.value)
    
    def is_number(self):
        return False
        
    def is_symbol(self):
        return False
    
    def could_extract_minus_sign(self):
        return False
    
    def __repr__(self):
        return str(self)
    
    def __hash__(self):
        return hash(str(self))  
    
    def __eq__(self, other):
        if not isinstance(other, PolynomialTerm):
            return False
        return str(self) == str(other)  

class Variable(PolynomialTerm):
    def __init__(self, value):
        self.value = value
        
    def is_symbol(self):
        return True
    
    def __pow__(self, exponent):
        if isinstance(exponent, Constant):
            return Power(self, exponent)
        elif isinstance(exponent, (int, float)):
            return Power(self, Constant(exponent))
        
    def __hash__(self):
        return hash(self.value)
    
    def __eq__(self, other):
        if not isinstance(other, Variable):
            return False
        return self.value == other.value
       


class Constant(PolynomialTerm):
    def __init__(self, value):
        self.value = float(value)
        
    def is_number(self):
        return True
        
    @property
    def is_integer(self):
        return self.value == int(self.value)
        
    def __gt__(self, second):
        if isinstance(second, Constant):
            return self.value > second.value
        return self.value > second
        
    def __eq__(self, second):
        if isinstance(second, Constant):
            return self.value == second.value
        return self.value == second
    
    def __hash__(self):
        return hash(self.value)
    
    def __mod__(self, other):
        if isinstance(other, Constant):
            return Constant(self.value % other.value)
        return Constant(self.value % other)

    def __rmod__(self, other):
        return Constant(other % self.value)
    
    def __sub__(self, other):
        if isinstance(other, Constant):
            return Constant(self.value - other.value)
        return Constant(self.value - other)

    def __rsub__(self, other):
        return Constant(other - self.value)
    
    def __pow__(self, exponent):
        if isinstance(exponent, Constant):
            return Constant(self.value ** exponent.value)
        return Constant(self.value ** exponent)
    
    def __floordiv__(self, other):
        if isinstance(other, Constant):
            return Constant(self.value // other.value)
        return Constant(self.value // other)

    def __rfloordiv__(self, other):
        return Constant(other // self.value)


class BinaryOperation(PolynomialTerm):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    @property
    def args(self):
        return [self.left, self.right]


class Addition(BinaryOperation):
    def __str__(self):
        return f"{self.left} + {self.right}"
    

class Subtraction(BinaryOperation):
    def __str__(self):
        return f"{self.left} - {self.right}"
    

class Multiplication(BinaryOperation):
    def __str__(self):
        # Adds parentheses around additions and subtractions
        left_str = str(self.left)
        right_str = str(self.right)

        if isinstance(self.left, (Addition, Subtraction)):
            left_str = f"({left_str})"
            
        if isinstance(self.right, (Addition, Subtraction)):
            right_str = f"({right_str})"
            
        return f"{left_str} * {right_str}"
    

class Power(BinaryOperation):
    def __str__(self):
        # Adds parentheses around the base for clarity
        left_str = str(self.left)
        if isinstance(self.left, (Addition, Subtraction, Multiplication)):
            left_str = f"({left_str})"
            
        return f"{left_str}^{self.right}"
    

class UnaryMinus(PolynomialTerm):
    def __init__(self, value):
        self.value = value
    
    def __str__(self):
        term_str = str(self.value)
        if isinstance(self.value, (Addition, Subtraction, Multiplication)):
            term_str = f"({term_str})"
        return f"-{term_str}"
    
    def is_number(self):
        return self.value.is_number() if hasattr(self.value, 'is_number') else False
    
    def could_extract_minus_sign(self):
        return True
    
    @property
    def args(self):
        return [self.value]


class PolynomialParser:
    def __init__(self, expr_string):
        self.expr_tree = None
        self.parse(expr_string)

    
    def parse(self, expr):
        self.expr = expr.replace(' ', '') 
        self.pos = 0
        self.expr_tree = self.parse_expression()
        return self
    
    def current_char(self):
        if self.pos >= len(self.expr):
            return None
        return self.expr[self.pos]
    
    def next_char(self):
        if self.pos >= len(self.expr):
            return None
        return self.expr[self.pos + 1] if self.pos + 1 < len(self.expr) else None

    
    def parse_expression(self):
        if self.current_char() == '-' and self.next_char() and not self.next_char().isdigit():
            self.pos += 1
            # left = Multiplication(Constant(-1.0), self.parse_term())

            left = UnaryMinus(self.parse_term())
            
        else:
            left = self.parse_term()
        
        while self.current_char() in ('+', '-'):
            op = self.current_char()
            self.pos += 1
            right = self.parse_term()
            
            if op == '+':
                left = Addition(left, right)
            else:
                left = Subtraction(left, right)
        
        return left
    
    def parse_term(self):
        left = self.parse_factor()
        
        while self.current_char() == '*':
            self.pos += 1
            right = self.parse_factor()
            left = Multiplication(left, right)
                
        return left
    
    def parse_factor(self):
        char = self.current_char()
        
        # End of expression check
        if char is None:
            return None
        
        #  unary minus
        if char == '-' and self.next_char() and not self.next_char().isdigit():
            self.pos += 1
            return UnaryMinus(self.parse_factor())
        
        # Brackets
        if char == '(':
            self.pos += 1
            expr = self.parse_expression()
            self.pos += 1  # Skip closing bracket
            
            if self.current_char() == '^':
                self.pos += 1
                exponent = self.parse_factor()
                return Power(expr, exponent)
                
            return expr
        
        #  numbers
        if char.isdigit() or (char == '-' and self.next_char() and self.next_char().isdigit()):
            return self.parse_number()
        
        #  variables
        if char.isalpha():
            var = self.parse_variable()
            
            # Check for power after variable
            if self.current_char() == '^':
                self.pos += 1
                exponent = self.parse_factor()
                return Power(var, exponent)

            return var
            
    
    def parse_number(self):
        start_pos = self.pos
        char = self.current_char()
        
        # Handle negative sign only at the beginning
        if char == '-':
            self.pos += 1
            char = self.current_char()
        
        # Parse digits and decimal point
        while char is not None and (char.isdigit() or char == '.'):
            self.pos += 1
            char = self.current_char()
        
        # Create constant
        value = float(self.expr[start_pos:self.pos])
        return Constant(value)
    
    def parse_variable(self):
        start_pos = self.pos
        char = self.current_char()
        
        while char is not None and char.isalnum():
            self.pos += 1
            char = self.current_char()
            
        variable = self.expr[start_pos:self.pos]
        return Variable(variable)
    

    def count(self, node=None):

        if node is None:
            node = self.expr_tree
        
        return self.count_terms(node)
    
    def count_terms(self, node):

        if type(node) in (Addition, Subtraction):
            left_count = self.count_terms(node.left)
            right_count = self.count_terms(node.right)
            return left_count + right_count
        else:
            return 1
        
    def find_variables(self, node = None, variables = None):

        if variables is None:
            variables = set()

        if node is None:
            node = self.expr_tree

        if isinstance(node, Variable):
            variables.add(node.value)

        elif isinstance(node, (Addition, Subtraction, Multiplication, Power)):
            self.find_variables(node.left, variables)
            self.find_variables(node.right, variables)
        elif isinstance(node, UnaryMinus):
            self.find_variables(node.value, variables)
    
        return variables
    

        
       


In [421]:
def extract_co_kernels(expression):
    kernels = []
    cokernels = []
    kernel_cokernel_pairs = []
    
    # Collects the terms from the expression
    terms_and_positions = group_terms(expression)
    terms = [term for term, pos in terms_and_positions]
    term_positions = {term: pos for term, pos in terms_and_positions}

    # print(term_positions)
    
    # For all terms find all possible factors
    all_factors = set()
    
    for term in terms:
        term_factors = extract_factors(term)
        all_factors.update(term_factors)

    # These are all the possible co-kernels
    all_factors = list(all_factors)

    print("All terms: ", terms)
    print("All factors: ", all_factors)
    
    # Keep track of unique pairs to avoid duplicates
    seen_pairs = set()
    
    # Go through all the factors and see if it can be factored out
    for potential_co_kernels in all_factors:
        kernel_terms = []
        factorisable_terms = []
        factored_positions = []
        
        for term in terms:
            cokernel = try_factorising(term, potential_co_kernels)
            if cokernel is not None:
                kernel_terms.append(cokernel)
                factorisable_terms.append(term)
                factored_positions.append(term_positions[term])

        # A valid kernel must be factorisable from at least 2 terms
        if len(kernel_terms) >= 2:
            # Sort kernel terms by position
            kernel_terms_ordered = list(zip(factored_positions, kernel_terms, factorisable_terms))
            kernel_terms_ordered.sort(key=lambda x: x[0]) 

            print(kernel_terms_ordered)

            # Orders the kernel terms in the order they appear in the expression
            # A seperate list stores the indices of the terms that are part of the kernel
            kernel_terms = [term for _, term, _ in kernel_terms_ordered]
            kernel_term_positions = [pos for pos, _, _ in kernel_terms_ordered]

            print(kernel_terms)
            print(kernel_term_positions)
           
            print(type(kernel_terms[0]))
            kernel_expr = kernel_terms[0]
            for i in range(1, len(kernel_terms)):
                if type(kernel_terms[i]) == UnaryMinus:
                    kernel_expr = Subtraction(kernel_expr, kernel_terms[i].value)
                else:
                    kernel_expr = Addition(kernel_expr, kernel_terms[i])
            
            print("Co kernel expression: ", kernel_expr)
            print("Co kernel positions: ", kernel_term_positions)
            # Create a normalized representation for duplicate detection
            kernel_str = str(potential_co_kernels)
            cokernel_str = str(kernel_expr)
            
            # Sort the factorisable terms to create a consistent signature
            term_signatures = sorted([str(term) for term in factorisable_terms])
            pair_signature = (kernel_str, cokernel_str, tuple(term_signatures))
            
            # Only add if we haven't seen this exact factorisation before
            if pair_signature not in seen_pairs:
                seen_pairs.add(pair_signature)
                kernel_cokernel_pairs.append({
                    'cokernel': potential_co_kernels,
                    'kernel': kernel_expr,
                    'kernel_pos': kernel_term_positions,
                    'original_terms': factorisable_terms,
                })
    
    return kernel_cokernel_pairs

def group_terms(expression, position = 0):
    # Goes through and groups terms seperated by subtraction and addition. These are the groups
    terms = []
    
    if type(expression) == Addition:
        left_terms = group_terms(expression.left, position)
        terms.extend(left_terms)

        right_position = position + len(left_terms)
        right_terms = group_terms(expression.right, right_position)

        terms.extend(right_terms)
    elif type(expression) == Subtraction:
        left_terms = group_terms(expression.left, position)
        terms.extend(left_terms)

        right_position = position + len(left_terms)
        right_terms = group_terms(expression.right, right_position)

        for term, pos in right_terms:
            terms.append((UnaryMinus(term), pos))
    else:
        terms.append((expression, position))
    
    return terms

def extract_factors(term):
    factors = set()
    
    if type(term) == Variable:
        factors.add(term)
    elif type(term) == Multiplication:
        # Get all individual factors first
        all_factors_list = flatten_multiplication_factors(term)
        
        # Add individual factors
        for factor in all_factors_list:
            if type(factor) != Constant:  # Don't add constants as potential co-kernels
                factors.add(factor)
        
        # Generate all possible combinations of factors (2 or more factors combined)
        from itertools import combinations
        for r in range(2, len(all_factors_list) + 1):
            for combo in combinations(all_factors_list, r):
                # Skip combinations that are only constants
                if all(type(f) == Constant for f in combo):
                    continue
                
                # Build multiplication from combination
                if len(combo) == 1:
                    factors.add(combo[0])
                else:
                    result = combo[0]
                    for factor in combo[1:]:
                        result = Multiplication(result, factor)
                    factors.add(result)
        
    elif type(term) == Power:
        # Add base, meaning to the power of one
        factors.add(term.left)

        # Add all other powers, starting from power of 2. 
        if type(term.right) == Constant and term.right.value >= 2:
            for i in range(2, int(term.right.value) + 1):
                factors.add(Power(term.left, Constant(i)))
    elif type(term) == UnaryMinus:
        # Extract factors from the inner term
        inner_factors = extract_factors(term.value)
        factors.update(inner_factors)

    return factors

def try_factorising(term, co_kernel):
    if equal_terms(term, co_kernel):
        return Constant(1.0)
    
    if co_kernel == Constant(1):
        return term

    
    elif type(term) == Multiplication:
        # For multiplication terms, we need to check if the co_kernel can be factored out
        # from the chain of multiplications
        term_factors = flatten_multiplication_factors(term)
        
        if type(co_kernel) == Multiplication:
            # Co-kernel is also a multiplication, need to check if all its factors
            # can be found in the term's factors
            cokernel_factors = flatten_multiplication_factors(co_kernel)
            remaining_factors = term_factors.copy()
            
            # Try to match each co-kernel factor with a term factor
            for ck_factor in cokernel_factors:
                matched = False
                for i, term_factor in enumerate(remaining_factors):
                    # Check for exact match
                    if equal_terms(term_factor, ck_factor):
                        remaining_factors.pop(i)
                        matched = True
                        break
                    # Check for power factorization (e.g., x^6 contains x^2)
                    elif (type(term_factor) == Power and type(ck_factor) == Power and
                          equal_terms(term_factor.left, ck_factor.left) and
                          type(term_factor.right) == Constant and type(ck_factor.right) == Constant and
                          term_factor.right.value > ck_factor.right.value):
                        # Replace with reduced power
                        new_power = term_factor.right.value - ck_factor.right.value
                        if new_power == 1:
                            remaining_factors[i] = term_factor.left
                        else:
                            remaining_factors[i] = Power(term_factor.left, Constant(new_power))
                        matched = True
                        break
                    # Check for power vs variable (e.g., x^6 contains x)
                    elif (type(term_factor) == Power and type(ck_factor) == Variable and
                          equal_terms(term_factor.left, ck_factor) and
                          type(term_factor.right) == Constant and term_factor.right.value > 1):
                        new_power = term_factor.right.value - 1
                        if new_power == 1:
                            remaining_factors[i] = term_factor.left
                        else:
                            remaining_factors[i] = Power(term_factor.left, Constant(new_power))
                        matched = True
                        break
                
                if not matched:
                    return None  # Couldn't factor out this co-kernel
            
            # If we get here, all co-kernel factors were successfully factored out
            if len(remaining_factors) == 0:
                return Constant(1.0)
            elif len(remaining_factors) == 1:
                return remaining_factors[0]
            else:
                # Rebuild multiplication from remaining factors
                result = remaining_factors[0]
                for factor in remaining_factors[1:]:
                    result = Multiplication(result, factor)
                return result
        
        else:
            # Co-kernel is a single factor
            for i, term_factor in enumerate(term_factors):
                # Check for exact match
                if equal_terms(term_factor, co_kernel):
                    remaining = term_factors[:i] + term_factors[i+1:]
                    if len(remaining) == 0:
                        return Constant(1.0)
                    elif len(remaining) == 1:
                        return remaining[0]
                    else:
                        result = remaining[0]
                        for factor in remaining[1:]:
                            result = Multiplication(result, factor)
                        return result
                
                # Check for power factorization
                elif (type(term_factor) == Power and type(co_kernel) == Power and
                      equal_terms(term_factor.left, co_kernel.left) and
                      type(term_factor.right) == Constant and type(co_kernel.right) == Constant and
                      term_factor.right.value > co_kernel.right.value):
                    new_power = term_factor.right.value - co_kernel.right.value
                    remaining = term_factors.copy()
                    if new_power == 1:
                        remaining[i] = term_factor.left
                    else:
                        remaining[i] = Power(term_factor.left, Constant(new_power))
                    
                    if len(remaining) == 1:
                        return remaining[0]
                    else:
                        result = remaining[0]
                        for factor in remaining[1:]:
                            result = Multiplication(result, factor)
                        return result
                
                # Check for power vs variable
                elif (type(term_factor) == Power and type(co_kernel) == Variable and
                      equal_terms(term_factor.left, co_kernel) and
                      type(term_factor.right) == Constant and term_factor.right.value > 1):
                    new_power = term_factor.right.value - 1
                    remaining = term_factors.copy()
                    if new_power == 1:
                        remaining[i] = term_factor.left
                    else:
                        remaining[i] = Power(term_factor.left, Constant(new_power))
                    
                    if len(remaining) == 1:
                        return remaining[0]
                    else:
                        result = remaining[0]
                        for factor in remaining[1:]:
                            result = Multiplication(result, factor)
                        return result
    
    elif type(term) == Power and type(co_kernel) == Power:
        # If term and co kernel are powers, than the exponent of the co kernel needs to be smaller than the term
        if (equal_terms(term.left, co_kernel.left) and term.right.value > co_kernel.right.value):
            # Factorise exponent 
            new_power = term.right.value - co_kernel.right.value
            if new_power == 1:
                return term.left
            else:
                return Power(term.left, Constant(new_power))
    
    # In the case where the co kernel does not have a power and has exponent of 1
    elif type(term) == Power:
        # Check if the bases match
        if equal_terms(term.left, co_kernel):
            if type(term.right) == Constant and term.right.value > 1:
                new_power = term.right.value - 1
                if new_power == 1:
                    return term.left
                else:
                    return Power(term.left, Constant(new_power))
    
    elif type(term) == UnaryMinus and type(co_kernel) != UnaryMinus:
        inner_kernel = try_factorising(term.value, co_kernel)
        if inner_kernel is not None:
            return UnaryMinus(inner_kernel)
        
    elif type(co_kernel) == UnaryMinus and type(term) != UnaryMinus:
        inner_kernel = try_factorising(term, co_kernel.value)
        if inner_kernel is not None:
            return UnaryMinus(inner_kernel)
    elif type(term) == UnaryMinus and type(co_kernel) == UnaryMinus:
        inner_kernel = try_factorising(term.value, co_kernel.value)
        # print(inner_kernel)
        return inner_kernel

    
    return None

def flatten_multiplication_factors(expr):

    if type(expr) != Multiplication:
        return [expr]
    
    factors = []
    factors.extend(flatten_multiplication_factors(expr.left))
    factors.extend(flatten_multiplication_factors(expr.right))
    return factors

def normalize_multiplication_factors(factors):

    def factor_sort_key(factor):
        if type(factor) == Constant:
            return (0, factor.value, 0)  # Added third element for consistency
        elif type(factor) == Variable:
            return (1, factor.value, 0)
        elif type(factor) == Power:
            if type(factor.left) == Variable and type(factor.right) == Constant:
                return (2, factor.left.value, factor.right.value)
            elif type(factor.left) == Variable:
                return (2, factor.left.value, float('inf'))
            else:
                # For complex power expressions, use string representation but ensure consistency
                return (3, str(factor), 0)
        else:
            # For other complex expressions
            return (4, str(factor), 0)
    
    return sorted(factors, key=factor_sort_key)

def equal_terms(term1, term2):
    if type(term1) != type(term2):
        return False
    
    if type(term1) == Variable:
        return term1.value == term2.value
    
    elif type(term1) == Constant:
        return (abs(term1.value - term2.value) < 1e-10)
    
    elif type(term1) == Multiplication:
        # Handle commutative multiplication by comparing normalized factor lists
        factors1 = flatten_multiplication_factors(term1)
        factors2 = flatten_multiplication_factors(term2)
        
        if len(factors1) != len(factors2):
            return False
        
        # Normalize and compare
        norm_factors1 = normalize_multiplication_factors(factors1)
        norm_factors2 = normalize_multiplication_factors(factors2)
        
        return all(equal_terms(f1, f2) for f1, f2 in zip(norm_factors1, norm_factors2))
    
    elif type(term1) in (Addition, Subtraction, Power):
        left_equals = equal_terms(term1.left, term2.left)
        right_equals = equal_terms(term1.right, term2.right)
        return (left_equals and right_equals)
    
    elif type(term1) == UnaryMinus:
        return equal_terms(term1.value, term2.value)

# Test code
test_expression = "(4*x^6*y*z) - (5*y*x^4*z^3) - (3*z^2*x^2) + (9*y*x^2*z^2)"

print(f"Expression: {test_expression}")
print("=" * 50)

parser = PolynomialParser(test_expression)
kernel_pairs = extract_co_kernels(parser.expr_tree)

print("Expression has length: ", parser.count())

for i, kernel_pair in enumerate(kernel_pairs):
    print(f"Pair {i}:")
    print(f"  Kernel:    {kernel_pair['kernel']}")
    print(f"  Co-kernel: {kernel_pair['cokernel']}")
    print(f"  Kernel Positions: {kernel_pair['kernel_pos']}")
    print("-" * 40)

Expression: (4*x^6*y*z) - (5*y*x^4*z^3) - (3*z^2*x^2) + (9*y*x^2*z^2)
All terms:  [4.0 * x^6.0 * y * z, -(5.0 * y * x^4.0 * z^3.0), -(3.0 * z^2.0 * x^2.0), 9.0 * y * x^2.0 * z^2.0]
All factors:  [9.0 * y * x^2.0, 4.0 * x^6.0, z^2.0, 5.0 * y * z^3.0, y * x^2.0, x^4.0 * z^3.0, 9.0 * y, 5.0 * z^3.0, 5.0 * y, 9.0 * x^2.0 * z^2.0, y * x^2.0 * z^2.0, 4.0 * x^6.0 * y * z, x^2.0 * z^2.0, 9.0 * y * x^2.0 * z^2.0, 9.0 * z^2.0, x^6.0, 3.0 * z^2.0 * x^2.0, 4.0 * x^6.0 * y, x^4.0, 4.0 * z, 5.0 * x^4.0 * z^3.0, z^3.0, 9.0 * y * z^2.0, 5.0 * y * x^4.0, 4.0 * y, y * z, 5.0 * x^4.0, x^2.0, 3.0 * z^2.0, x^6.0 * y, 5.0 * y * x^4.0 * z^3.0, y * x^4.0 * z^3.0, 4.0 * x^6.0 * z, 3.0 * x^2.0, y * z^2.0, 9.0 * x^2.0, y * z^3.0, z, x^6.0 * y * z, 4.0 * y * z, y, x^6.0 * z, y * x^4.0, z^2.0 * x^2.0]
[(1, -(5.0 * y * x^4.0 * z), -(5.0 * y * x^4.0 * z^3.0)), (2, -(3.0 * x^2.0), -(3.0 * z^2.0 * x^2.0)), (3, 9.0 * y * x^2.0, 9.0 * y * x^2.0 * z^2.0)]
[-(5.0 * y * x^4.0 * z), -(3.0 * x^2.0), 9.0 * y * x^2.0]
[1, 2, 3

In [422]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np


def generate_polynomial_matrix(expression, df,  initial_terms, prev_cokernel = None, prev_df = None):

    print(prev_df)

    initial_terms = [term for (term, _) in initial_terms]

    print(initial_terms)
    # Initialize data structures
    variables = []
    terms = []
    signs = []
    
    # Extract terms from the expression
    parser = PolynomialParser("")
    parser.expr_tree = expression

    # term_breakdown = parser.get_term_breakdown()
    # terms = term_breakdown['terms']

    # print(term_breakdown['terms'])
        

    grouped_terms = group_terms(parser.expr_tree)

    terms = [m[0] for m in grouped_terms]

    print("Term order: ", terms)
    # Find all unique variables in the expression
    variables = list(parser.find_variables())
    # variables.sort()  # Sort for consistent ordering
    
    # Build the matrix data
    matrix_data = []
    for i, term in enumerate(terms):
        term_row, term_sign = analyse_term(term, variables)
        term_row['Sign'] = term_sign
        term_row['prev_cokernel'] = prev_cokernel
        if prev_cokernel == None:
            term_row['expr_pos'] = i
        else:
            term_row['expr_pos'] = np.nan

        matrix_data.append(term_row)
        signs.append(term_sign)
    

    # print(matrix_data)
    # Create pandas DataFrame
    # Convert new data to DataFrame
    new_df = pd.DataFrame(matrix_data)




    for var in variables:
        # print(var)
        new_df[var] = new_df[var].astype(int)

    # Reorder columns to have variables first, then sign
    column_order = variables + ['Sign'] + ['prev_cokernel'] + ['expr_pos']
    new_df = new_df[column_order]

    #df['Terms'] = [term for term in terms]
    new_df.insert(loc = 0, column = 'Terms', value = [term for term in terms])
    #display(new_df)
    
    # print([str(term) for term in terms])

    # print(new_df.iloc[3]['Terms'])
    # print(initial_terms)
    # print(prev_cokernel)
    # for item in initial_terms:
    #     answer = try_factorising(item,new_df.iloc[3]['Terms'] )
    #     print(answer)
    #     test = flatten_multiplication_factors(answer)
    #     print(type(test[0]))
    #     print(flatten_multiplication_factors(prev_cokernel))
    #     print(set(flatten_multiplication_factors(answer)) == set(flatten_multiplication_factors(prev_cokernel)))
    
    # # print(initial_terms)
    # print("new df terms:")
    # print(new_df['Terms'])
    if prev_df is not None:

        for original in initial_terms:
            for item in new_df['Terms']:
                for prev in prev_df['Terms']:
                    #print( original, term, prev)
                    prev_prev_co_kernel = prev_df.loc[prev_df['Terms'] == prev, 'prev_cokernel'].values[0]
                    if prev_prev_co_kernel is not None:

                        flattened_original_new = flatten_multiplication_factors(try_factorising(original, item))
                        flattened_original_prev = flatten_multiplication_factors(try_factorising(original, prev))

                        flattened_prev_cokernel = flatten_multiplication_factors(prev_cokernel)
                        flattened_prev_prev_cokernel = flatten_multiplication_factors(prev_prev_co_kernel)

                        
                        if set(flattened_original_new) == set(flattened_prev_cokernel) and set(flattened_original_prev) == set(flattened_prev_prev_cokernel):
                            print(flattened_prev_cokernel, flattened_prev_prev_cokernel)
                            print(flattened_original_new, flattened_original_prev)
                            print("HEREHRHRHEHR")
                            new_df.loc[new_df['Terms'] == item, 'expr_pos'] = prev_df.loc[prev_df['Terms'] == prev, 'expr_pos'].values[0]
                            print(prev_df.loc[prev_df['Terms'] == prev, 'expr_pos'].values[0])
                    
                    
                    
                    else:
                        #print(item, prev)
                        flattened_original_new = flatten_multiplication_factors(try_factorising(original, item))


                        if set(flattened_original_new) == set(flatten_multiplication_factors(prev_cokernel)) and set(flatten_multiplication_factors(prev)) == set(flatten_multiplication_factors(original)):
                            print(item)
                            print("HEIKFGJGKRKGKG")
                            new_df.loc[new_df['Terms'] == item, 'expr_pos'] = prev_df.loc[prev_df['Terms'] == prev, 'expr_pos'].values[0]
                            print(prev_df.loc[prev_df['Terms'] == prev, 'expr_pos'].values[0])



    
    df = pd.concat([df, new_df], ignore_index=True)
    
    return df, variables

# def find_all_variables(node):
    
#     variables = set()
    
#     if isinstance(node, Variable):
#         variables.add(node.value)
#     elif isinstance(node, (Addition, Subtraction, Multiplication, Power)):
#         variables.update(find_all_variables(node.left))
#         variables.update(find_all_variables(node.right))
#     elif isinstance(node, UnaryMinus):
#         variables.update(find_all_variables(node.value))


#     return list(variables)

def analyse_term(term, variables):
   
    powers = {var: 0 for var in variables}
    sign = 1
    
    # Handle negative terms
    if isinstance(term, UnaryMinus):
        sign = -1
        term = term.value
    
    # Extract variable powers from the term
    var_powers = extract_powers(term)
    
    # Update the powers dictionary
    for var_name, power in var_powers.items():
        if var_name in powers:
            powers[var_name] = power

    return powers, sign


def extract_powers(node):

    var_powers = {}
    
    if isinstance(node, Variable):
        var_powers[node.value] = 1

   
    elif isinstance(node, Multiplication):
        # Combine powers from both sides
        left_powers = extract_powers(node.left)
        right_powers = extract_powers(node.right)
        
        # Merge the dictionaries, adding powers for same variables
        for var, power in left_powers.items():
           
            if var in var_powers:
                var_powers[var] = var_powers + power
            else:
                var_powers[var] = power

        for var, power in right_powers.items():
            if var in var_powers:
                var_powers[var] = var_powers + power
            else:
                var_powers[var] = power
            
    elif isinstance(node, Power):
        # Get the base variable powers and multiply by exponent
        base_powers = extract_powers(node.left)
        
      
        exp_value = int(node.right.value)

        for var, power in base_powers.items():
            var_powers[var] = power * exp_value

        
    # elif isinstance(node, UnaryMinus):
    #     # Extract from the inner term (sign handled elsewhere)
    #     var_powers = extract_powers(node.value)
    
    return var_powers


def largest_factor(df, variables):

    variables_df = df[df.columns[df.columns.isin(variables)]]

    #print(variables_df)

    biggest_factor = None


    # Find column (variable) with the least number of zeros
    zero_counts = (variables_df == 0).sum(axis=0)

    # Find the minimum zero count
    min_zero_count = zero_counts.min()

    # Select columns with the minimum zero count
    candidate_cols = zero_counts[zero_counts == min_zero_count].index

    # Among these candidates, select the column with the highest average. Highest average means it is the column with
    # most exponents
    col_avgs = variables_df[candidate_cols].mean(axis=0)
    selected_column = col_avgs.idxmax()

    # Df with only the terms that include the most appearing variable
    filtered_rows = variables_df[variables_df[selected_column] != 0]

    #Remove columns (variables) that contain zeros in the filtered df. These are the variables that cannot be factorised
    # The result are the term numbers and the variables that can be factored out with their power
    filtered_no_zero_cols = filtered_rows.loc[:, ~(filtered_rows == 0).any()]

    # print(filtered_no_zero_cols)



    print("===========================")
    for i in range (0, len(filtered_no_zero_cols.columns)):

        # print(variables_df.columns[i], "non zero")
        power = filtered_no_zero_cols.iloc[:, i].min()
        # print("Power", power)

        if power > 1:

            factor_term = Power(Variable(filtered_no_zero_cols.columns[i]), Constant(power))
        else:
            factor_term = Variable(filtered_no_zero_cols.columns[i])

        if biggest_factor is None:
            biggest_factor = factor_term
        else:
            biggest_factor = Multiplication(biggest_factor, factor_term)
    

    print("Biggest factor is: ", biggest_factor)
    # print(len(variables_df.columns))
    # print(variables_df.columns[0])
    return biggest_factor




# print(f"\n{'='*80}")
# print(f"Expression: {test_expression}")
# print('='*80)

# # Parse the expression
# parser = PolynomialParser(test_expression)

# matrix, variables = generate_polynomial_matrix(parser.expr_tree, expression_matrix)
# display(matrix)

# co_kernel = largest_factor(matrix, variables)


def combine_multiplications(mult1, mult2):

    factors1 = flatten_multiplication_factors(mult1)
    factors2 = flatten_multiplication_factors(mult2)
    

    variable_powers = {}
    other_factors = []
    
    all_factors = factors1 + factors2
    
    for factor in all_factors:
        if type(factor) == Variable:

            if factor.value in variable_powers:
                variable_powers[factor.value] += 1
            else:
                variable_powers[factor.value] = 1
                
        elif type(factor) == Power and type(factor.left) == Variable and type(factor.right) == Constant:

            var_name = factor.left.value
            power = factor.right.value
            if var_name in variable_powers:
                variable_powers[var_name] += power
            else:
                variable_powers[var_name] = power
                
        else:
            
            other_factors.append(factor)
    
    result_factors = []
    
    # Add variable factors with their combined powers
    for var_name, total_power in variable_powers.items():
        if total_power == 1:
            result_factors.append(Variable(var_name))
        else:
            result_factors.append(Power(Variable(var_name), Constant(total_power)))
    

    result_factors.extend(other_factors)
    
   
    if len(result_factors) == 1:
        return result_factors[0]
    else:
        result = result_factors[0]
        for factor in result_factors[1:]:
            result = Multiplication(result, factor)
        return result
    


def add_kernel_cokernel_pair(parsed_expression, co_kernel, kernel_cokernel_pairs, expression_matrix):
    one_term = False
    terms_and_positions = group_terms(parsed_expression)

    


    terms = [term for term, pos in terms_and_positions]
    term_positions = {term: pos for term, pos in terms_and_positions}

    # print(terms)
    # print(term_positions)

    kernel_terms = []
    factorisable_terms = []
    factored_positions = []

    for term in terms:
        # print(term)
        kernel = try_factorising(term, co_kernel)
        # print(cokernel)
        if kernel is not None:
            kernel_terms.append(kernel)
            factorisable_terms.append(term)
            factored_positions.append(term_positions[term])


    # print("kernel terms: ", kernel_terms)


    kernel_terms_ordered = list(zip(factored_positions, kernel_terms, factorisable_terms))
    kernel_terms_ordered.sort(key=lambda x: x[0]) 

    # print(kernel_terms_ordered)

    # Orders the kernel terms in the order they appear in the expression
    # A seperate list stores the indices of the terms that are part of the kernel
    kernel_terms = [term for _, term, _ in kernel_terms_ordered]
    kernel_term_positions = [pos for pos, _, _ in kernel_terms_ordered]


    if not expression_matrix['prev_cokernel'].isnull().all():

        prev_cokernels = expression_matrix[['Terms', 'prev_cokernel']].dropna()

        str_factorisable_terms = [str(item) for item in factorisable_terms]
        prev_cokernels = prev_cokernels[prev_cokernels["Terms"].isin(str_factorisable_terms)]

        print(prev_cokernels)

        prev_cokernel = (prev_cokernels['prev_cokernel'].iloc[0])
        # print((prev_cokernels['prev_cokernel'][0]))
        # print("Current cokernel: ", co_kernel)
        # print(kernel_term_positions)

        # If there is only one potential kernel term, it should not be added as a kernel, co kernel pair
        if len(prev_cokernels) <= 1:
            one_term = True
            print("Kernel only has 1 term")


        co_kernel = combine_multiplications(prev_cokernel, co_kernel)



    # print(type(kernel_terms[0]))
    kernel_expr = kernel_terms[0]
    for i in range(1, len(kernel_terms)):
        if type(kernel_terms[i]) == UnaryMinus:
            kernel_expr = Subtraction(kernel_expr, kernel_terms[i].value)
        else:
            kernel_expr = Addition(kernel_expr, kernel_terms[i])

    print("Co kernel expression: ", co_kernel )
    print("Kernel expression: ", kernel_expr)
    print("Kernel positions: ", kernel_term_positions)

    new_terms_and_positions = group_terms(kernel_expr)
    new_terms = [term for term, pos in new_terms_and_positions]
    new_term_positions = {term: pos for term, pos in new_terms_and_positions}

    if not one_term:
        kernel_cokernel_pairs.append({
            'cokernel': co_kernel,
            'kernel': kernel_expr,
            'kernel_pos': kernel_term_positions,
            'original_terms': factorisable_terms,
            'new_terms': new_terms,
        })

    for_next_pair = {
        'cokernel': co_kernel,
        'kernel': kernel_expr,
        'kernel_pos': kernel_term_positions,
        'original_terms': factorisable_terms,
        'new_terms': new_terms,
    }

    return kernel_cokernel_pairs, for_next_pair



def find_all_kernel_cokernel_pairs(test_expression):

    print(f"\n{'='*80}")
    print(f"Expression: {test_expression}")
    print('='*80)

    kernel_cokernel_pairs = []
    expression_matrix = pd.DataFrame()
    
    # Parse initial expression
    parser = PolynomialParser(test_expression)
    current_expression = parser.expr_tree
    
    iteration = 0
    max_iterations = 20  # Safety limit to prevent infinite loops

    initial_terms = group_terms(current_expression)
    
    while iteration < max_iterations:
        print(f"\n--- ITERATION {iteration + 1} ---")
        
        # Generate polynomial matrix for current expression

       
        expression_matrix, variables = generate_polynomial_matrix(
            current_expression, 
            expression_matrix,
            initial_terms,
            prev_cokernel=kernel_cokernel_pairs[-1]['cokernel'] if kernel_cokernel_pairs else None,
            prev_df = previous_expression_matrix if kernel_cokernel_pairs else None
        )
        
        print("Current expression matrix:")
        display(expression_matrix)
        previous_expression_matrix = expression_matrix.copy()


        # Check if we can find a common factor
        if expression_matrix.empty:
            print("Expression matrix is empty. No more factorizations possible.")
            break
        
        expression_matrix["Terms"] = expression_matrix["Terms"].apply(str)


        co_kernel = largest_factor(expression_matrix, variables)
        if co_kernel is None:
            print("No common factor found. Factorization complete.")
            break
    
        
        kernel_cokernel_pairs, info_for_next_pair = add_kernel_cokernel_pair(
            current_expression, 
            co_kernel, 
            kernel_cokernel_pairs, 
            expression_matrix
        )
        
        # Check if factorization was successful
        if info_for_next_pair is None:
            print("No terms could be factorized.")
            break
        
        
        # Remove factorized terms from the matrix
        str_original_terms = [str(term) for term in info_for_next_pair['original_terms']]
        expression_matrix = expression_matrix[~expression_matrix['Terms'].isin(str_original_terms)]
        
        if expression_matrix.empty:
            expression_matrix = pd.DataFrame()
        
        print("Updated expression matrix after removing factorized terms:")
        display(expression_matrix)
        
        # Check if kernel has more than one term for further factorization
        kernel_terms_and_positions = group_terms(info_for_next_pair['kernel'])
        if len(kernel_terms_and_positions) <= 1:
            print("Kernel has only one term. No further factorization possible.")
            break
        
        # Set up for next iteration
        current_expression = info_for_next_pair['kernel']
        iteration += 1
    
    if iteration >= max_iterations:
        print(f"Warning: Reached maximum iterations ({max_iterations}). Stopping to prevent infinite loop.")
    
    return kernel_cokernel_pairs


def display_factorization_results(kernel_cokernel_pairs):

    print(f"\n{'='*80}")
    print("FACTORIZATION RESULTS")
    print('='*80)
    
    
    print(f"Found {len(kernel_cokernel_pairs)} kernel-cokernel pair(s):\n")
    
    for i, pair in enumerate(kernel_cokernel_pairs, 1):
        print(f"Pair {i}:")
        print(f"  Cokernel: {pair['cokernel']}")
        print(f"  Kernel:   {pair['kernel']}")
        print(f"  Original terms factorized: {len(pair['original_terms'])}")
        print(f"  Position terms in kernel: {(pair['kernel_pos'])}")
        print()
    



test_expression = "-(4*x^6*y*z^2) - (5*y*x^4*z^3) - (3*z^4*y) + (5*x^2*y^2) + (9*y*x^2*z)"
# test_expression = "x - 3*x^3 + 5*x^5 - 7*x^7"

# Find all kernel-cokernel pairs
all_pairs = find_all_kernel_cokernel_pairs(test_expression)

# Display results
display_factorization_results(all_pairs)






Expression: -(4*x^6*y*z^2) - (5*y*x^4*z^3) - (3*z^4*y) + (5*x^2*y^2) + (9*y*x^2*z)

--- ITERATION 1 ---
None
[-(4.0 * x^6.0 * y * z^2.0), -(5.0 * y * x^4.0 * z^3.0), -(3.0 * z^4.0 * y), 5.0 * x^2.0 * y^2.0, 9.0 * y * x^2.0 * z]
Term order:  [-(4.0 * x^6.0 * y * z^2.0), -(5.0 * y * x^4.0 * z^3.0), -(3.0 * z^4.0 * y), 5.0 * x^2.0 * y^2.0, 9.0 * y * x^2.0 * z]
Current expression matrix:


,Terms,x,z,y,Sign,prev_cokernel,expr_pos
0,-(4.0 * x^6.0 * y * z^2.0),6,2,1,-1,None,0
1,-(5.0 * y * x^4.0 * z^3.0),4,3,1,-1,None,1
2,-(3.0 * z^4.0 * y),0,4,1,-1,None,2
3,5.0 * x^2.0 * y^2.0,2,0,2,1,None,3
4,9.0 * y * x^2.0 * z,2,1,1,1,None,4


Biggest factor is:  y
Co kernel expression:  y
Kernel expression:  -(4.0 * x^6.0 * z^2.0) - 5.0 * x^4.0 * z^3.0 - 3.0 * z^4.0 + 5.0 * x^2.0 * y + 9.0 * x^2.0 * z
Kernel positions:  [0, 1, 2, 3, 4]
Updated expression matrix after removing factorized terms:


""



--- ITERATION 2 ---
                        Terms  x  z  y  Sign prev_cokernel  expr_pos
0  -(4.0 * x^6.0 * y * z^2.0)  6  2  1    -1          None         0
1  -(5.0 * y * x^4.0 * z^3.0)  4  3  1    -1          None         1
2          -(3.0 * z^4.0 * y)  0  4  1    -1          None         2
3         5.0 * x^2.0 * y^2.0  2  0  2     1          None         3
4         9.0 * y * x^2.0 * z  2  1  1     1          None         4
[-(4.0 * x^6.0 * y * z^2.0), -(5.0 * y * x^4.0 * z^3.0), -(3.0 * z^4.0 * y), 5.0 * x^2.0 * y^2.0, 9.0 * y * x^2.0 * z]
Term order:  [-(4.0 * x^6.0 * z^2.0), -(5.0 * x^4.0 * z^3.0), -(3.0 * z^4.0), 5.0 * x^2.0 * y, 9.0 * x^2.0 * z]
-(4.0 * x^6.0 * z^2.0)
HEIKFGJGKRKGKG
0
-(5.0 * x^4.0 * z^3.0)
HEIKFGJGKRKGKG
1
-(3.0 * z^4.0)
HEIKFGJGKRKGKG
2
5.0 * x^2.0 * y
HEIKFGJGKRKGKG
3
9.0 * x^2.0 * z
HEIKFGJGKRKGKG
4
Current expression matrix:


...


--- ITERATION 6 ---
                Terms  x  z    y  Sign      prev_cokernel  expr_pos
0      -(3.0 * z^4.0)  0  4  0.0    -1                  y       2.0
1             5.0 * y  0  0  1.0     1          y * x^2.0       3.0
2                 9.0  0  0  NaN     1      y * x^2.0 * z       4.0
3      -(4.0 * x^4.0)  4  0  NaN    -1  y * x^2.0 * z^2.0       0.0
4  -(5.0 * x^2.0 * z)  2  1  NaN    -1  y * x^2.0 * z^2.0       1.0
[-(4.0 * x^6.0 * y * z^2.0), -(5.0 * y * x^4.0 * z^3.0), -(3.0 * z^4.0 * y), 5.0 * x^2.0 * y^2.0, 9.0 * y * x^2.0 * z]
Term order:  [-(4.0 * x^2.0), -(5.0 * z)]
[y, x^4.0, z^2.0] [y, x^2.0, z^2.0]
[x^4.0, y, z^2.0] [x^2.0, y, z^2.0]
HEREHRHRHEHR
0.0
[y, x^4.0, z^2.0] [y, x^2.0, z^2.0]
[y, x^4.0, z^2.0] [y, x^2.0, z^2.0]
HEREHRHRHEHR
1.0
Current expression matrix:


,Terms,x,z,y,Sign,prev_cokernel,expr_pos
0,-(3.0 * z^4.0),0,4,0.0,-1,y,2.0
1,5.0 * y,0,0,1.0,1,y * x^2.0,3.0
2,9.0,0,0,NaN,1,y * x^2.0 * z,4.0
3,-(4.0 * x^2.0),2,0,NaN,-1,y * x^4.0 * z^2.0,0.0
4,-(5.0 * z),0,1,NaN,-1,y * x^4.0 * z^2.0,1.0


Biggest factor is:  z
        Terms      prev_cokernel
4  -(5.0 * z)  y * x^4.0 * z^2.0
Kernel only has 1 term
Co kernel expression:  y * x^4.0 * z^3.0
Kernel expression:  -5.0
Kernel positions:  [1]
Updated expression matrix after removing factorized terms:


,Terms,x,z,y,Sign,prev_cokernel,expr_pos
0,-(3.0 * z^4.0),0,4,0.0,-1,y,2.0
1,5.0 * y,0,0,1.0,1,y * x^2.0,3.0
2,9.0,0,0,NaN,1,y * x^2.0 * z,4.0
3,-(4.0 * x^2.0),2,0,NaN,-1,y * x^4.0 * z^2.0,0.0


Kernel has only one term. No further factorization possible.

FACTORIZATION RESULTS
Found 5 kernel-cokernel pair(s):

Pair 1:
  Cokernel: y
  Kernel:   -(4.0 * x^6.0 * z^2.0) - 5.0 * x^4.0 * z^3.0 - 3.0 * z^4.0 + 5.0 * x^2.0 * y + 9.0 * x^2.0 * z
  Original terms factorized: 5
  Position terms in kernel: [0, 1, 2, 3, 4]

Pair 2:
  Cokernel: y * x^2.0
  Kernel:   -(4.0 * x^4.0 * z^2.0) - 5.0 * x^2.0 * z^3.0 + 5.0 * y + 9.0 * z
  Original terms factorized: 4
  Position terms in kernel: [0, 1, 3, 4]

Pair 3:
  Cokernel: y * x^2.0 * z
  Kernel:   -(4.0 * x^4.0 * z) - 5.0 * x^2.0 * z^2.0 + 9.0
  Original terms factorized: 3
  Position terms in kernel: [0, 1, 3]

Pair 4:
  Cokernel: y * x^2.0 * z^2.0
  Kernel:   -(4.0 * x^4.0) - 5.0 * x^2.0 * z
  Original terms factorized: 2
  Position terms in kernel: [0, 1]

Pair 5:
  Cokernel: y * x^4.0 * z^2.0
  Kernel:   -(4.0 * x^2.0) - 5.0 * z
  Original terms factorized: 2
  Position terms in kernel: [0, 1]

